# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [3]:
import pandas as pd

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [4]:
from sklearn.model_selection import train_test_split

RANDOM = 43

In [5]:
X = df.drop('salary_in_usd', axis=1)
y = df['salary_in_usd']

# Может вызывать утечку, так как это косвенная (а в некоторых случаях и прямая, когда зарплата сразу указана в долларах) ссылка на наш таргет
X = X.drop('salary', axis=1)

x_train_val, x_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=RANDOM)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=1/9, random_state=RANDOM)

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time

In [7]:
categorical_features = ['experience_level', 'employment_type', 'job_title',
                        'salary_currency', 'employee_residence', 'company_location', 'company_size']

# Пора работать по-взролому: с пайплайнами
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], remainder='passthrough')

lr = Pipeline(steps=[('preprocessor', preprocessor),
                     ('regressor', LinearRegression())])


lr.fit(x_train, y_train)

y_pred = lr.predict(x_val)

# Вычисление MAPE и RMSE
mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print('MAPE: {:.2f}%'.format(mape * 100))
print('RMSE: ', rmse)

MAPE: 37.19%
RMSE:  49622.0475923838


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [8]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [9]:
params = {
    'xgb_regressor__max_depth' : [1, 3, 5, 7],
    'xgb_regressor__learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'xgb_regressor__n_estimators' : [50, 100, 150, 200],
    'xgb_regressor__gamma' : [0, 0.1, 0.2]
}

xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb_regressor', XGBRegressor(
        objective='reg:squarederror',
        max_depth=1,
        learning_rate=0.01,
        n_estimators=50,
        gamma=0
    ))
])

grid_search = GridSearchCV(xgb, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1)

grid_search.fit(x_train, y_train)

print('Best parameters:', grid_search.best_params_)

Fitting 3 folds for each of 240 candidates, totalling 720 fits
Best parameters: {'xgb_regressor__gamma': 0, 'xgb_regressor__learning_rate': 0.2, 'xgb_regressor__max_depth': 3, 'xgb_regressor__n_estimators': 200}


In [10]:
best_params_reformatted = {key.split('__')[-1]: value for key, value in grid_search.best_params_.items()}

xgb_best = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb_regressor', XGBRegressor(
        objective='reg:squarederror',
        **best_params_reformatted
    ))
])

# Learning time
start_time = time.time()
xgb_best.fit(x_train, y_train)
training_time = time.time() - start_time

# Prediction time
start_time = time.time()
y_pred = xgb_best.predict(x_val)
prediction_time = time.time() - start_time

mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print('MAPE: {:.2f}%'.format(mape * 100))
print('RMSE:', rmse)
print('Training time:', training_time, 'seconds')
print('Prediction time:', prediction_time, 'seconds')

MAPE: 33.90%
RMSE: 48498.350112846754
Training time: 0.12026095390319824 seconds
Prediction time: 0.010742902755737305 seconds


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [11]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.3 MB/s eta 0:00:00


In [12]:
from catboost import CatBoostRegressor

In [13]:
params = {
    'catboost_regressor__depth': [1, 3, 5, 7],
    'catboost_regressor__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'catboost_regressor__iterations': [50, 100, 150, 200]
}

catboost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('catboost_regressor', CatBoostRegressor(
        loss_function='RMSE',
        verbose=False
    ))
])

grid_search = GridSearchCV(catboost, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(x_train, y_train)

print('Best parameters:', grid_search.best_params_)

Fitting 3 folds for each of 80 candidates, totalling 240 fits
Best parameters: {'catboost_regressor__depth': 5, 'catboost_regressor__iterations': 200, 'catboost_regressor__learning_rate': 0.15}


In [14]:
best_params_reformatted = {key.split('__')[-1]: value for key, value in grid_search.best_params_.items()}

catboost_best = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('catboost_regressor', CatBoostRegressor(
        **best_params_reformatted,
        loss_function='RMSE',
        verbose=False
    ))
])

start_time = time.time()
catboost_best.fit(x_train, y_train)
training_time = time.time() - start_time

start_time = time.time()
y_pred = catboost_best.predict(x_val)
prediction_time = time.time() - start_time

mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print('MAPE: {:.2f}%'.format(mape * 100))
print('RMSE:', rmse)
print('Training time:', training_time, 'seconds')
print('Prediction time:', prediction_time, 'seconds')

MAPE: 33.40%
RMSE: 49005.67815616109
Training time: 0.4817543029785156 seconds
Prediction time: 0.018001317977905273 seconds


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [21]:
from catboost import Pool, cv
import numpy as np

In [23]:
categorical_features_indices = [x_train.columns.get_loc(c) for c in categorical_features if c in x_train]

train_pool = Pool(data=x_train, label=y_train, cat_features=categorical_features_indices)
val_pool = Pool(data=x_val, label=y_val, cat_features=categorical_features_indices)

depths = [1, 3, 5, 7]
learning_rates = [0.01, 0.05, 0.1, 0.2]
iterations_list = [50, 100, 150, 200]

best_score = float('inf')
best_params = {}

# GreedSearch никак не хотел работать тут, поэтому циклы
for depth in depths:
    for learning_rate in learning_rates:
        for iterations in iterations_list:
            params = {
                'depth': depth,
                'learning_rate': learning_rate,
                'iterations': iterations,
                'loss_function': 'RMSE',
                'verbose': False
            }

            cv_results = cv(
                pool=train_pool,
                params=params,
                fold_count=3,
                type='Classical',
                verbose=False
            )

            mean_cv_score = np.mean(cv_results['test-RMSE-mean'])

            if mean_cv_score < best_score:
                best_score = mean_cv_score
                best_params = params.copy()

print('Best parameters:', best_params)
print('Best CV score:', best_score)

Training on fold [0/3]

bestTest = 101562.123
bestIteration = 49

Training on fold [1/3]

bestTest = 102554.6437
bestIteration = 49

Training on fold [2/3]

bestTest = 105256.9155
bestIteration = 49

Training on fold [0/3]

bestTest = 75051.01922
bestIteration = 99

Training on fold [1/3]

bestTest = 77048.62285
bestIteration = 99

Training on fold [2/3]

bestTest = 80013.64121
bestIteration = 99

Training on fold [0/3]

bestTest = 61815.59595
bestIteration = 149

Training on fold [1/3]

bestTest = 63123.93782
bestIteration = 149

Training on fold [2/3]

bestTest = 66331.92638
bestIteration = 149

Training on fold [0/3]

bestTest = 55861.24247
bestIteration = 199

Training on fold [1/3]

bestTest = 57098.24558
bestIteration = 199

Training on fold [2/3]

bestTest = 59666.22442
bestIteration = 199

Training on fold [0/3]

bestTest = 52892.73176
bestIteration = 49

Training on fold [1/3]

bestTest = 54375.53955
bestIteration = 49

Training on fold [2/3]

bestTest = 56264.94433
bestIterat

In [24]:
cbr = CatBoostRegressor(
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    iterations=best_params['iterations'],
    loss_function='RMSE',
    verbose=False
)

start_training = time.time()
cbr.fit(train_pool)
end_training = time.time()

start_prediction = time.time()
y_pred = cbr.predict(val_pool)
end_prediction = time.time()

mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print('MAPE: {:.2f}%'.format(mape * 100))
print('RMSE:', rmse)
print('Training time:', end_training - start_training, 'seconds')
print('Prediction time:', end_prediction - start_prediction, 'seconds')

MAPE: 32.77%
RMSE: 48161.85027475245
Training time: 1.6494977474212646 seconds
Prediction time: 0.0017542839050292969 seconds


## Ответ
У меня не стало сильно лучше по качеству( А вот время обучения увеличилось в разы

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [28]:
from lightgbm import LGBMRegressor

categorical_features = ['experience_level', 'employment_type', 'job_title',
                        'salary_currency', 'employee_residence', 'company_location', 'company_size']
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', 'passthrough', numeric_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lgbmregressor', LGBMRegressor())
])

params = {
    'lgbmregressor__learning_rate': [0.01, 0.05, 0.1],
    'lgbmregressor__n_estimators': [100, 200, 300],
    'lgbmregressor__max_depth': [10, 20, 30]
}

grid_search = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(x_train, y_train)


print('Best parameters:', grid_search.best_params_)
best_params = {key.replace('lgbmregressor__', ''): value for key, value in grid_search.best_params_.items()}

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 80
[LightGBM] [Info] Number of data points in the train set: 2002, number of used features: 38
[LightGBM] [Info] Start training from score 136813.444555
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] 

In [30]:
lgbm_best = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lgbmregressor', LGBMRegressor(**best_params))
])

start_time = time.time()
lgbm_best.fit(x_train, y_train)
training_time = time.time() - start_time

start_time = time.time()
y_pred = lgbm_best.predict(x_val)
prediction_time = time.time() - start_time

mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print(f'MAPE: {mape * 100:.2f}%')
print(f'RMSE: {rmse}')
print(f'Training time: {training_time} seconds')
print(f'Prediction time: {prediction_time} seconds')

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000962 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 3003, number of used features: 42
[LightGBM] [Info] Start training from score 137576.376623
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

## Ответ
| Модель    | Лучшие параметры                                                                      | MAPE  | RMSE         | Время обучения (сек) | Время предсказания (сек) |
|-----------|---------------------------------------------------------------------------------------|-------|--------------|---------------------|--------------------------|
| XGBoost   | gamma: 0, learning_rate: 0.2, max_depth: 3, n_estimators: 200                        | 33.90 | 48,498.35    | 0.120               | 0.011                    |
| CatBoost  | depth: 5, iterations: 200, learning_rate: 0.15                                       | 33.40 | 49,005.68    | 0.482               | 0.018                    |
| LightGBM  | learning_rate: 0.05, max_depth: 10, n_estimators: 100                               | 33.16 | 49,123.71    | 0.346               | 0.016                    |

**MAPE:** LightGBM показал лучшую производительность с MAPE 33.16%, затем следует CatBoost с 33.40% и XGBoost с 33.90%.

**RMSE:** Аналогично, LightGBM показал чуть лучший RMSE по сравнению с XGBoost, но немного хуже, чем CatBoost.

**Время обучения:** XGBoost оказался самой быстрой моделью для обучения, занимая всего около 0.12 секунд. За ним следует LightGBM с временем обучения 0.346 секунды, и CatBoost был самым медленным с 0.482 секунды.

**Время предсказания:** XGBoost также имел самое быстрое время предсказания, около 0.011 секунды, за ним следует LightGBM с 0.016 секунды и CatBoost с 0.018 секунды.

### Что из всего это следует?

**XGBoost** предпочитает более высокую скорость обучения и умеренное количество оценщиков, то есть он быстрее сходится, что круто.

**CatBoost** балансирует умеренную глубину и количество итераций с довольно агрессивной скоростью обучения.

**LightGBM** использовал меньшее количество оценщиков и более низкую скорость обучения, что может свидетельствовать о фокусе на стабильности модели и обобщении за счет более медленного обучения.

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [ ]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [ ]:
# -- YOUR CODE HERE --

Выкиньте строку под названием `user`.

In [ ]:
# -- YOUR CODE HERE --

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [ ]:
# -- YOUR CODE HERE --
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
ben harper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [ ]:
from sklearn.cluster import KMeans

# -- YOUR CODE HERE --

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --